In [ ]:
#install the required packages in Google Colab by running the following code
!pip install yfinance
# !pip install pandas
# !pip install numpy
# !pip install matplotlib

In [91]:
# Import necessary packages
import yfinance as yf
import pandas as pd
import numpy as np
from Symbols import nasdaq_symbols, SP_Symbols, DOWjones_symbols
import matplotlib.pyplot as plt

In [93]:
def download_index_data(symbols, start_date, end_date):
    """
    Download historical data for a list of symbols for multiple indices.

    Parameters:
    symbols (dict): A dictionary of index names and their respective symbol lists
    start_date (str): The start date for the data download in 'YYYY-MM-DD' format
    end_date (str): The end date for the data download in 'YYYY-MM-DD' format

    Returns:
    dict: A dictionary of index names and their respective price data as pandas DataFrames.
    """
    index_data = {}
    
    for index_name, index_symbols in symbols.items():
        print(f"Downloading data for {index_name}...")
        
        tickers = yf.download(index_symbols, start=start_date, end=end_date)
        prices = tickers['Adj Close']
        
        index_data[index_name] = prices
        
    return index_data


In [146]:
def get_top10_performers(index_name, index_prices, monthly_periods, quarterly_periods, annual_periods):
    # Create an empty dataframe to store the top performers
    top10_performers_df = pd.DataFrame(columns=["Pfolio_Pick_Period", "Performance_Eval_Period", "Ticker", "Return"])

    # Iterate over the periods in the timeframe and calculate the returns
    for periods, name in zip([monthly_periods, quarterly_periods, annual_periods], ["Monthly", "Quarterly", "Annual"]):
        print(f"Top 10 performers in {name} timeframe:")
        for i, period in enumerate(periods[:-1]):
            # Select the data for the period
            start_date = period
            end_date = periods[i+1]

            # Calculate the midpoint between the start and end dates
            midpoint_date = start_date + (end_date - start_date) / 2

            # Select the data for the portfolio pick period
            pick_data = index_prices.loc[start_date:midpoint_date]

            # Check that the pick_data has at least two rows
            if len(pick_data) < 2:
                continue

            # Calculate the returns for each stock in the portfolio pick period
            pick_returns = pick_data.pct_change() + 1
            pick_returns = pick_returns.iloc[-1].dropna()

            # Sort the returns and select the top 10 performers
            top_performers = pick_returns.nlargest(10)

            # Select the data for the performance evaluation period
            eval_data = index_prices.loc[midpoint_date + pd.DateOffset(days=1):end_date]

            # Calculate the returns for each stock in the performance evaluation period
            eval_returns = eval_data.pct_change().iloc[-1].dropna() + 1

            # Filter the top performers to only include stocks that are in the performance evaluation period
            top_performers = top_performers[top_performers.index.isin(eval_returns.index)]

            # Add the top performers to the dataframe
            period_df = pd.DataFrame({
                "Pfolio_Pick_Period": start_date.date(),
                "Performance_Eval_Period": end_date.date(),
                "Ticker": top_performers.index,
                "Return": eval_returns[top_performers.index].values
            })
            top10_performers_df = pd.concat([top10_performers_df, period_df], ignore_index=True)

    # Set the Performance_Eval_Period column as the index column
    top10_performers_df.set_index('Performance_Eval_Period', inplace=True)

    # # Add the index name as the main header of the dataframe
    # top10_performers_df.columns = pd.MultiIndex.from_tuples([(index_name, col) for col in top10_performers_df.columns])

    return top10_performers_df


In [467]:
def get_top10_by_tf(index_name, index_prices, monthly_periods, quarterly_periods, annual_periods):
    top10_performers = {}

    # Iterate over the periods in the timeframe and calculate the returns
    for periods, name in zip([monthly_periods, quarterly_periods, annual_periods], ["Monthly", "Quarterly", "Annual"]):
        print(f"Top 10 performers in {name} timeframe:")
        top10_performers_df = pd.DataFrame(columns=["Pick_Period", "Eval_Period", "Ticker", "Return"])
        for i, period in enumerate(periods[:-1]):
            # Select the data for the period
            start_date = period
            end_date = periods[i+1]

            # Calculate the midpoint between the start and end dates
            midpoint_date = start_date + (end_date - start_date) / 2

            # Select the data for the portfolio pick period
            pick_data = index_prices.loc[start_date:midpoint_date]

            # Check that the pick_data has at least two rows
            if len(pick_data) < 2:
                continue

            # Calculate the returns for each stock in the portfolio pick period
            pick_returns = pick_data.pct_change() + 1
            pick_returns = pick_returns.iloc[-1].dropna()

            # Sort the returns and select the top 10 performers
            top_performers = pick_returns.nlargest(10)

            # Select the data for the performance evaluation period
            eval_data = index_prices.loc[midpoint_date + pd.DateOffset(days=1):end_date]

            # Calculate the returns for each stock in the performance evaluation period
            eval_returns = eval_data.pct_change().iloc[-1].dropna() + 1

            # Filter the top performers to only include stocks that are in the performance evaluation period
            top_performers = top_performers[top_performers.index.isin(eval_returns.index)]

            # Add the top performers to the dataframe
            period_df = pd.DataFrame({
                "Pick_Period": start_date.date(),
                "Eval_Period": end_date.date(),
                "Ticker": top_performers.index,
                "Return": eval_returns[top_performers.index].values
            })
            top10_performers_df = pd.concat([top10_performers_df, period_df], ignore_index=True)

        # Set the Performance_Eval_Period column as the index column
        top10_performers_df.set_index('Eval_Period', inplace=True)

        top10_performers[name] = top10_performers_df

    return top10_performers


In [309]:
def calculate_cumulative_returns(index_name, df, Freq):
    # Calculate the mean returns for each year
    mean_returns = df.groupby(pd.Grouper(level=0, freq='Y')).mean()
    mean_returns = mean_returns.reset_index().rename(columns={'level_0': 'Year', 'Return': 'Mean Return'})

    # Calculate the cumulative returns
    cumulative_returns = (1 + mean_returns['Mean Return']).cumprod() - 1
    cumulative_returns = cumulative_returns.to_frame().rename(columns={'Mean Return': 'Cumulative Return'})
    print(f"{index_name} {Freq} Cumulative Returns:\n{cumulative_returns}")
    # Combine the mean returns and cumulative returns into a single dataframe
    result_df = pd.concat([mean_returns, cumulative_returns], axis=1)
    # result_df = result_df.set_index(('Pfolio_Pick_Period', 'Performance_Eval_Period'))
    
    return result_df

In [291]:
def plot_cumulative_returns1(df):
    # Call the calculate_cumulative_returns function to calculate the cumulative returns
    cumulative_returns = calculate_cumulative_returns(df)
    # print(cumulative_returns)
    # Set the performance_eval_period column as the index
    cumulative_returns = cumulative_returns.set_index('Performance_Eval_Period')
    
    # Plot the cumulative returns
    cumulative_returns['Cumulative Return'].plot(figsize=(10, 5), title="Cumulative Returns", xlabel="Year", ylabel="Cumulative Returns")


    # Show the plot
    plt.show()


In [418]:
def cal_cumulative_returns(df, freq):
    # Convert index to a datetime index
    df.index = pd.to_datetime(df.index)
    # Group the data by similar dates in the index column as one portfolio
    df_grouped = df.groupby(pd.Grouper(freq=freq)).mean()
    # Calculate the mean returns for each portfolio
    mean_returns = df_grouped.groupby(df_grouped.index.year).mean()
    # Add a new column for the performance evaluation period
    mean_returns['Performance_Eval_Period'] = mean_returns.index
    # Return the dataframe
    return mean_returns


In [419]:
def plot_cumulative_returns(index_name, annual_df, monthly_df, quarterly_df):
    # Calculate the cumulative returns for all three dataframes
    annual_cumulative_returns = cal_cumulative_returns(annual_df, 'Y')
    monthly_cumulative_returns = cal_cumulative_returns(monthly_df, 'M')
    quarterly_cumulative_returns = cal_cumulative_returns(quarterly_df, 'Q')
    
    # Set the 'Performance_Eval_Period' column as the index for all three dataframes
    annual_cumulative_returns = annual_cumulative_returns.set_index('Performance_Eval_Period')
    monthly_cumulative_returns = monthly_cumulative_returns.set_index('Performance_Eval_Period')
    quarterly_cumulative_returns = quarterly_cumulative_returns.set_index('Performance_Eval_Period')
    
    # Plot the cumulative returns for all three dataframes
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.plot(annual_cumulative_returns.index, annual_cumulative_returns[index_name], label='Annual')
    ax.plot(monthly_cumulative_returns.index, monthly_cumulative_returns[index_name], label='Monthly')
    ax.plot(quarterly_cumulative_returns.index, quarterly_cumulative_returns[index_name], label='Quarterly')
    ax.legend(loc='upper left')
    ax.set_title(f'Cumulative Returns of {index_name}')
    ax.set_xlabel('Performance Evaluation Period')
    ax.set_ylabel('Cumulative Returns')
    plt.show()


In [344]:
def calculate_cumulative_ret(index_name, df):
    # Group the data by the year
    df_grouped = df.groupby(pd.Grouper(level='Performance_Eval_Period', freq='Y'))
    
    # Calculate the mean returns for each year and portfolio
    mean_returns = df_grouped.mean()
    
    # Calculate the cumulative returns for the entire period
    cumulative_returns = (1 + mean_returns).cumprod() - 1
    
    # Combine the mean returns and cumulative returns into a single dataframe
    result_df = pd.concat([mean_returns, cumulative_returns], axis=1)
    result_df.columns = pd.MultiIndex.from_tuples([('Year', ''), ('Cumulative Return', 'NASDAQ')])
    result_df[('Mean Return', 'NASDAQ')] = mean_returns.values
    
    # Add index name to the title of the chart
    title = f"Cumulative Returns for {index_name}"
    
    return result_df, title


In [461]:
def calc_cumulative_ret(df):
    # Calculate mean returns for each unique date in Performance_Eval_Period
    portfolio = df.groupby(df.index).mean()
    print(portfolio)
        
    # Calculate cumulative product of mean returns
    cumulative_return = (1 + portfolio).cumprod().iloc[-1]
    
    return cumulative_returns

In [584]:
def compute_returns(df, freq):
    """
    Compute the mean and cumulative returns for a given dataframe
    using the resampling method with the specified frequency.

    Parameters:
    df (pandas.DataFrame): A dataframe containing the portfolio returns
    freq (str): A string specifying the resampling frequency, e.g. 'M' for monthly,
                'Q' for quarterly, or 'Y' for yearly.

    Returns:
    tuple: A tuple containing the mean and cumulative returns as pandas.Series.
    """
    df.index = pd.to_datetime(df.index)
    mean_returns = df['Return'].resample(freq).mean()
    cum_returns = (mean_returns + 1).cumprod() - 1
    return mean_returns, cum_returns


In [353]:
#Call the download_index_data function to download data for each index
index_data = download_index_data(symbols, start_date, end_date)
nasdaq_prices = index_data['NASDAQ']
sp_prices = index_data['S&P 500']
dow_prices = index_data['Dow Jones']

[*********************100%***********************]  101 of 101 completed
[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
- BRK.B: No timezone found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted
[*********************100%***********************]  30 of 30 completed


In [578]:
nasdaq_prices.pct_change()

,AAPL,ABNB,ADBE,ADI,ADP,ADSK,AEP,ALGN,AMAT,AMD,...,TSLA,TXN,VRSK,VRTX,WBA,WBD,WDAY,XEL,ZM,ZS
Date,,,,,,,,,,,,,,,,,,,,,
2012-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-02,0.031682,NaN,0.017516,0.046125,0.025646,0.044979,0.022727,0.023423,0.034965,0.054167,...,0.043992,0.045970,0.039435,0.041766,0.028371,0.031506,-0.029174,0.025084,NaN,NaN
2013-01-03,-0.012622,NaN,-0.015389,-0.016136,0.003939,-0.015430,-0.000687,0.027113,-0.002534,-0.015810,...,-0.016685,-0.012999,0.011514,0.034364,-0.007094,0.000764,-0.011529,-0.002922,NaN,NaN
2013-01-04,-0.027855,NaN,0.010066,-0.017787,0.008870,-0.000275,-0.001605,0.004799,0.000000,0.040161,...,-0.010642,-0.002195,-0.001679,0.015504,-0.016142,0.008393,-0.002103,0.003663,NaN,NaN
2013-01-07,-0.005882,NaN,-0.004983,0.003057,-0.003889,-0.007426,-0.004133,-0.012624,-0.011854,0.030888,...,-0.001744,0.003143,-0.004486,0.031407,0.022861,-0.002421,-0.000383,-0.010584,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23,-0.002798,0.004477,0.005735,0.000550,0.007443,0.000319,0.012914,0.000885,-0.003893,0.010335,...,-0.017551,-0.001943,0.006476,-0.014392,0.004943,-0.006500,-0.005857,0.012852,-0.000758,-0.005304
2022-12-27,-0.013878,-0.020645,-0.009928,-0.010010,0.000374,-0.009938,0.006270,0.004517,-0.019338,-0.019374,...,-0.114089,-0.003650,-0.000285,-0.008181,-0.008284,0.006543,-0.007532,0.009023,-0.006526,-0.011123
2022-12-28,-0.030685,-0.011977,-0.020174,-0.011837,-0.013193,-0.023565,-0.009658,-0.014712,-0.011642,-0.011064,...,0.033089,-0.015814,-0.007576,-0.005429,-0.019055,-0.039003,-0.002509,-0.007203,-0.001528,0.002696


In [577]:
nasdaq_prices

,AAPL,ABNB,ADBE,ADI,ADP,ADSK,AEP,ALGN,AMAT,AMD,...,TSLA,TXN,VRSK,VRTX,WBA,WBD,WDAY,XEL,ZM,ZS
Date,,,,,,,,,,,,,,,,,,,,,
2012-12-31,16.320076,NaN,37.680000,33.243801,39.790409,35.349998,29.587566,27.750000,9.775301,2.400000,...,2.258000,23.503288,49.553116,41.900002,27.634058,32.437405,54.500000,19.240644,NaN,NaN
2013-01-02,16.837124,NaN,38.340000,34.777168,40.810860,36.939999,30.260012,28.400000,10.117093,2.530000,...,2.357333,24.583731,51.507236,43.650002,28.418053,33.459377,52.910000,19.723284,NaN,NaN
2013-01-03,16.624599,NaN,37.750000,34.215992,40.971615,36.369999,30.239210,29.170000,10.091459,2.490000,...,2.318000,24.264162,52.100285,45.150002,28.216461,33.484924,52.299999,19.665648,NaN,NaN
2013-01-04,16.161526,NaN,38.130001,33.607391,41.335049,36.360001,30.190680,29.309999,10.091459,2.590000,...,2.293333,24.210901,52.012787,45.849998,27.761002,33.765968,52.189999,19.737688,NaN,NaN
2013-01-07,16.066460,NaN,37.939999,33.710133,41.174294,36.090000,30.065897,28.940001,9.971830,2.670000,...,2.289333,24.286987,51.779453,47.290001,28.395653,33.684212,52.169998,19.528782,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23,131.658981,85.250000,338.450012,163.081024,239.583176,188.160004,94.811234,203.669998,96.992851,64.519997,...,123.150002,163.216843,175.280060,289.690002,38.125446,9.170000,164.639999,69.885384,65.889999,108.779999
2022-12-27,129.831772,83.489998,335.089996,161.448624,239.672668,186.289993,95.405724,204.589996,95.117241,63.270000,...,109.099998,162.621094,175.230148,287.320007,37.809628,9.230000,163.399994,70.515953,65.459999,107.570000
2022-12-28,125.847855,82.489998,328.329987,159.537521,236.510574,181.899994,94.484268,201.580002,94.009842,62.570000,...,112.709999,160.049423,173.902649,285.760010,37.089161,8.870000,162.990005,70.008034,65.360001,107.860001


In [355]:
# Define the timeframes
monthly_periods = pd.date_range(start=start_date, end=end_date, freq="M")
quarterly_periods = pd.date_range(start=start_date, end=end_date, freq="3M")
annual_periods = pd.date_range(start=start_date, end=end_date, freq="Y")

In [356]:
quarterly_periods

DatetimeIndex(['2012-12-31', '2013-03-31', '2013-06-30', '2013-09-30',
               '2013-12-31', '2014-03-31', '2014-06-30', '2014-09-30',
               '2014-12-31', '2015-03-31', '2015-06-30', '2015-09-30',
               '2015-12-31', '2016-03-31', '2016-06-30', '2016-09-30',
               '2016-12-31', '2017-03-31', '2017-06-30', '2017-09-30',
               '2017-12-31', '2018-03-31', '2018-06-30', '2018-09-30',
               '2018-12-31', '2019-03-31', '2019-06-30', '2019-09-30',
               '2019-12-31', '2020-03-31', '2020-06-30', '2020-09-30',
               '2020-12-31', '2021-03-31', '2021-06-30', '2021-09-30',
               '2021-12-31', '2022-03-31', '2022-06-30', '2022-09-30',
               '2022-12-31'],
              dtype='datetime64[ns]', freq='3M')

In [357]:
nasdaq_top10_performers = get_top10_performers("NASDAQ", nasdaq_prices, monthly_periods, quarterly_periods, annual_periods)
sp500_top10_performers = get_top10_performers("S&P 500", sp_prices, monthly_periods, quarterly_periods, annual_periods)
dowjones_top10_performers = get_top10_performers("Dow Jones Industrial Average", dow_prices, monthly_periods, quarterly_periods, annual_periods)

Top 10 performers in Monthly timeframe:
Top 10 performers in Quarterly timeframe:
Top 10 performers in Annual timeframe:
Top 10 performers in Monthly timeframe:
Top 10 performers in Quarterly timeframe:
Top 10 performers in Annual timeframe:
Top 10 performers in Monthly timeframe:
Top 10 performers in Quarterly timeframe:
Top 10 performers in Annual timeframe:


In [548]:
top10_performers_dict = get_top10_by_tf("NASDAQ", nasdaq_prices, monthly_periods, quarterly_periods, annual_periods)

# Get the monthly dataframe
nasdaq_monthly_df = top10_performers_dict['Monthly']

# Get the quarterly dataframe
nasdaq_quarterly_df = top10_performers_dict['Quarterly']

# Get the annual dataframe
nasdaq_annual_df = top10_performers_dict['Annual']


Top 10 performers in Monthly timeframe:
Top 10 performers in Quarterly timeframe:
Top 10 performers in Annual timeframe:


In [581]:
nasdaq_annual_df.head(10)

,Pick_Period,Ticker,Return
Eval_Period,,,
2013-12-31,2012-12-31,TSLA,0.986815
2013-12-31,2012-12-31,NFLX,1.003215
2013-12-31,2012-12-31,SGEN,0.991549
2013-12-31,2012-12-31,INTU,0.996735
2013-12-31,2012-12-31,AAPL,1.011722
2013-12-31,2012-12-31,FANG,1.018294
2013-12-31,2012-12-31,ODFL,1.001890
2013-12-31,2012-12-31,REGN,0.989787
2013-12-31,2012-12-31,CTSH,1.008892


In [277]:
top10_performers_dict = get_top10_by_tf("S&P 500", sp_prices, monthly_periods, quarterly_periods, annual_periods)

# Get the monthly dataframe
sp500_monthly_df = top10_performers_dict['Monthly']

# Get the quarterly dataframe
sp500_quarterly_df = top10_performers_dict['Quarterly']

# Get the annual dataframe
sp500_annual_df = top10_performers_dict['Annual']


Top 10 performers in Monthly timeframe:
Top 10 performers in Quarterly timeframe:
Top 10 performers in Annual timeframe:


In [587]:
top10_performers_dict = get_top10_by_tf("Dow Jones Industrial Average", dow_prices, monthly_periods, quarterly_periods, annual_periods)

# Get the monthly dataframe
dj_monthly_df = top10_performers_dict['Monthly']

# Get the quarterly dataframe
dj_quarterly_df = top10_performers_dict['Quarterly']

# Get the annual dataframe
dj_annual_df = top10_performers_dict['Annual']


Top 10 performers in Monthly timeframe:
Top 10 performers in Quarterly timeframe:
Top 10 performers in Annual timeframe:


In [549]:
nasdaq_quarterly_df

,Pick_Period,Ticker,Return
Eval_Period,,,
2013-03-31,2012-12-31,ENPH,0.965732
2013-03-31,2012-12-31,NVDA,1.014229
2013-03-31,2012-12-31,BKR,0.998924
2013-03-31,2012-12-31,META,0.980452
2013-03-31,2012-12-31,FANG,1.027959
...,...,...,...
2022-12-31,2022-09-30,WDAY,0.997259
2022-12-31,2022-09-30,TEAM,0.992289
2022-12-31,2022-09-30,CRWD,1.006885


In [528]:
nasdaq_quarterly_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 400 entries, 2013-03-31 to 2022-12-31
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Pick_Period  400 non-null    object 
 1   Ticker       400 non-null    object 
 2   Return       400 non-null    float64
dtypes: float64(1), object(2)
memory usage: 12.5+ KB


In [553]:
nasdaq_quarterly_df.head(10)

,Pick_Period,Ticker,Return
Eval_Period,,,
2013-03-31,2012-12-31,ENPH,0.965732
2013-03-31,2012-12-31,NVDA,1.014229
2013-03-31,2012-12-31,BKR,0.998924
2013-03-31,2012-12-31,META,0.980452
2013-03-31,2012-12-31,FANG,1.027959
2013-03-31,2012-12-31,ATVI,1.006215
2013-03-31,2012-12-31,MU,1.002008
2013-03-31,2012-12-31,MCHP,1.007397
2013-03-31,2012-12-31,MRVL,1.006661


In [585]:
# Compute the mean and cumulative returns with monthly frequency for Nasdaq 
nasdaq_monthly_mean, nasdaq_monthly_cum = compute_returns(nasdaq_monthly_df, 'M')
nasdaq_quarterly_mean, nasdaq_quarterly_cum = compute_returns(nasdaq_quarterly_df, 'Q')
nasdaq_yearly_mean, nasdaq_yearly_cum = compute_returns(nasdaq_annual_df, 'Y')

In [586]:
nasdaq_monthly_cum

Eval_Period
2013-01-31    1.027576e+00
2013-02-28    3.090989e+00
2013-03-31    7.193625e+00
2013-04-30    1.542588e+01
2013-05-31    3.174874e+01
                  ...     
2022-08-31    8.472411e+34
2022-09-30    1.685687e+35
2022-10-31    3.358529e+35
2022-11-30    6.858251e+35
2022-12-31    1.371604e+36
Freq: M, Name: Return, Length: 120, dtype: float64

In [589]:
# Compute the mean and cumulative returns with monthly frequency for S&P 500 Index
sp500_monthly_mean, sp500_monthly_cum = compute_returns(sp500_monthly_df, 'M')
sp500_quarterly_mean, sp500_quarterly_cum = compute_returns(sp500_quarterly_df, 'Q')
sp500_yearly_mean, sp500_yearly_cum = compute_returns(sp500_annual_df, 'Y')

In [588]:
# Compute the mean and cumulative returns with monthly frequency for DOWjones Index
DJ_monthly_mean, DJ_monthly_cum = compute_returns(dj_monthly_df, 'M')
DJ_quarterly_mean, DJ_quarterly_cum = compute_returns(dj_quarterly_df, 'Q')
DJ_yearly_mean, DJ_yearly_cum = compute_returns(dj_annual_df, 'Y')

In [559]:
# group the data by the date in the Eval_Period column
grouped = nasdaq_annual_df.groupby('Eval_Period')

# calculate the mean return for each ticker on each date
mean_returns = grouped['Return'].mean()


In [569]:
nasdaq_monthly_df.index = pd.to_datetime(nasdaq_monthly_df.index)
mean_returns = nasdaq_monthly_df['Return'].resample('M').mean()

In [572]:
mean_returns

Eval_Period
2013-01-31    1.027576
2013-02-28    1.017674
2013-03-31    1.002847
2013-04-30    1.004715
2013-05-31    0.993728
                ...   
2022-08-31    0.998254
2022-09-30    0.989619
2022-10-31    0.992380
2022-11-30    1.042040
2022-12-31    0.999932
Freq: M, Name: Return, Length: 120, dtype: float64

In [543]:
mean_returns

Eval_Period
2013-03-31    1.001743
2013-06-30    0.999345
2013-09-30    0.995085
2013-12-31    1.004702
2014-03-31    1.013522
2014-06-30    1.009101
2014-09-30    0.994048
2014-12-31    0.996847
2015-03-31    0.992710
2015-06-30    1.002874
2015-09-30    1.021074
2015-12-31    0.992906
2016-03-31    1.006873
2016-06-30    1.014224
2016-09-30    1.017459
2016-12-31    0.987102
2017-03-31    1.003082
2017-06-30    0.995140
2017-09-30    1.008273
2017-12-31    0.987160
2018-03-31    1.025543
2018-06-30    0.994322
2018-09-30    1.008140
2018-12-31    1.003238
2019-03-31    0.999388
2019-06-30    1.001450
2019-09-30    1.010818
2019-12-31    1.004891
2020-03-31    0.997044
2020-06-30    1.027286
2020-09-30    1.007187
2020-12-31    0.993503
2021-03-31    1.027802
2021-06-30    1.006828
2021-09-30    1.002499
2021-12-31    0.989507
2022-03-31    0.978233
2022-06-30    0.980660
2022-09-30    0.986778
2022-12-31    0.993463
Name: Return, dtype: float64

In [573]:
cumulative_return = (1 + mean_returns).cumprod() - 1

In [574]:
cumulative_return

Eval_Period
2013-01-31    1.027576e+00
2013-02-28    3.090989e+00
2013-03-31    7.193625e+00
2013-04-30    1.542588e+01
2013-05-31    3.174874e+01
                  ...     
2022-08-31    8.472411e+34
2022-09-30    1.685687e+35
2022-10-31    3.358529e+35
2022-11-30    6.858251e+35
2022-12-31    1.371604e+36
Freq: M, Name: Return, Length: 120, dtype: float64

In [544]:
mean_returns.head(40)

Eval_Period
2013-03-31    1.001743
2013-06-30    0.999345
2013-09-30    0.995085
2013-12-31    1.004702
2014-03-31    1.013522
2014-06-30    1.009101
2014-09-30    0.994048
2014-12-31    0.996847
2015-03-31    0.992710
2015-06-30    1.002874
2015-09-30    1.021074
2015-12-31    0.992906
2016-03-31    1.006873
2016-06-30    1.014224
2016-09-30    1.017459
2016-12-31    0.987102
2017-03-31    1.003082
2017-06-30    0.995140
2017-09-30    1.008273
2017-12-31    0.987160
2018-03-31    1.025543
2018-06-30    0.994322
2018-09-30    1.008140
2018-12-31    1.003238
2019-03-31    0.999388
2019-06-30    1.001450
2019-09-30    1.010818
2019-12-31    1.004891
2020-03-31    0.997044
2020-06-30    1.027286
2020-09-30    1.007187
2020-12-31    0.993503
2021-03-31    1.027802
2021-06-30    1.006828
2021-09-30    1.002499
2021-12-31    0.989507
2022-03-31    0.978233
2022-06-30    0.980660
2022-09-30    0.986778
2022-12-31    0.993463
Name: Return, dtype: float64

In [484]:
cumulative_return = (1 + mean_returns).cumprod() - 1

In [486]:
cumulative_return

Eval_Period
2013-01-31    1.027576e+00
2013-02-28    3.090989e+00
2013-03-31    7.193625e+00
2013-04-30    1.542588e+01
2013-05-31    3.174874e+01
                  ...     
2022-08-31    8.472411e+34
2022-09-30    1.685687e+35
2022-10-31    3.358529e+35
2022-11-30    6.858251e+35
2022-12-31    1.371604e+36
Name: Return, Length: 120, dtype: float64